In [47]:
import pandas as pd
from gurobipy import *
from pyomo.environ import *

In [48]:
xls = pd.ExcelFile('Proyecto_1_TransporteMaritimo_Dataset2.xlsx', engine='openpyxl')
nombres= xls.sheet_names
df1 = pd.read_excel(xls, nombres[0])
df2 = pd.read_excel(xls, nombres[1])
df3= pd.read_excel(xls, nombres[2])
df4= pd.read_excel(xls, nombres[3])
df5= pd.read_excel(xls, nombres[4])

Se transforman los datos de los dataframes a diccionarios para el procesamiento de datos

In [49]:
barcos_sheet = df1.to_dict("index")
compatibilidad_sheet = df2.to_dict()
cargos_sheet = df3.to_dict("index")
costos_transporte_sheet = df4.to_dict("index")
costos_transporte_dict = {(costos_transporte_sheet[i]['ID_Barco'],costos_transporte_sheet[i]['Puerto_Origen'],costos_transporte_sheet[i]['Puerto_Destino']) : {'Tiempo_Viaje_hrs':costos_transporte_sheet[i]['Tiempo_Viaje_hrs'], 'Costo_Viaje_libras':costos_transporte_sheet[i]['Costo_Viaje_libras']} for i in costos_transporte_sheet.keys()}
costos_tiempo_sheet = df5.to_dict("index")
costos_tiempo_dict = {(costos_tiempo_sheet[i]['ID_Barco'],costos_tiempo_sheet[i]['ID_Cargo']) : {'Tiempo_Origen_hrs':costos_tiempo_sheet[i]['Tiempo_Origen_hrs'], 'Costos_Origen_hrs':costos_tiempo_sheet[i]['Costos_Origen_hrs'], 'Tiempo_Destino_hrs':costos_tiempo_sheet[i]['Tiempo_Destino_hrs'], 'Costos_Destino_hrs':costos_tiempo_sheet[i]['Costos_Destino_hrs']} for i in costos_tiempo_sheet.keys()}

In [50]:
n_cargos = len([cargos_sheet[i]['ID_Cargo'] for i in cargos_sheet.keys() if cargos_sheet[i]['ID_Cargo']>=0])

 La manera en que funcionan los nodos es: los primeros $n$ ($n =$ número de cargos) son los nodos de carga, los siguientes $n$ son los nodos de descarga, los siguientes $v$ ($v =$ número de barcos) son los nodos de origen, y los último v son los nodos de destino.

In [51]:
V = [1+i for i in barcos_sheet.keys()] ## conjunto barcos
n_barcos = len(V)

In [52]:
Kv= {} # conjunto capacidades de barcos
for i in barcos_sheet.keys():
    Kv[barcos_sheet[i]['ID_Barco']] =  barcos_sheet[i]['Capacidad']

In [53]:
# N^P
nodos_carga= [cargos_sheet[i]['ID_Cargo'] for i in cargos_sheet.keys() if cargos_sheet[i]['ID_Cargo']>=0]

#N^D
nodos_descarga= [i+len(nodos_carga) for i in range(1,len(nodos_carga)+1)] ##conjunto nodos de descarga

In [54]:
nodos = {}
for i in range(n_cargos):
    nodos[cargos_sheet[i]['ID_Cargo']] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Origen'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Carga'],
                                          'RT': cargos_sheet[i]['RT_Carga']}
    
    nodos[cargos_sheet[i]['ID_Cargo']+n_cargos] = {'ID_Cargo': cargos_sheet[i]['ID_Cargo'],
                                          'ID_Puerto': cargos_sheet[i]['ID_Puerto_Destino'],
                                          'Tamano': cargos_sheet[i]['Tamano'],
                                          'Costo_SPOT': cargos_sheet[i]['Costo_SPOT'],
                                          'LT': cargos_sheet[i]['LT_Descarga'],
                                          'RT': cargos_sheet[i]['RT_Descarga']}
o = {}
d = {}
for v in V:
    o[v] = n_cargos*2+v
    nodos[n_cargos*2+v] = {'Costo_SPOT': 0,
                           'ID_Cargo': 0,
                           'ID_Puerto': barcos_sheet[v-1]['ID_Puerto '],
                           'LT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'RT': barcos_sheet[v-1]['Tiempo_Inicio'],
                           'Tamano': 0}
    d[v] = n_cargos*2+n_barcos+v
    nodos[n_cargos*2+n_barcos+v] = {'Costo_SPOT': 0,
                                    'ID_Cargo': 0,
                                    'ID_Puerto': 0,
                                    'LT': 100000,
                                    'RT': 100000,
                                    'Tamano': 0}

In [55]:
compatibilidad = {}
for i in V:
    compatibilidad[i] = [int(j) for j in list(df2.loc[i-1])[1:] if j>0]
compatibilidad

Nv = {}
for i in V:
    Nv[i] = []
    for j in nodos.keys():
        if nodos[j]["ID_Cargo"] in compatibilidad[i] or (j == i+n_cargos*2 or j==i+n_cargos*2+len(V)): 
            Nv[i].append(j)

In [56]:
"""import itertools
Av = {}
for v in V:
    Av[v] = list(itertools.permutations(Nv[v],2))
"""
Av={}
indices = []
for v in V:
    Av[v] = [] 
    for i in Nv[v]:
        for j in Nv[v]:
                Av[v].append((i,j))
                indices.append((i,j,v))
indices = tuplelist(indices)

In [57]:
NP_v = {}
for v in V:
    NP_v[v] = [i for i in nodos_carga if i in Nv[v]]

ND_v = {}
for v in V:
    ND_v[v] = [i for i in nodos_descarga if i in Nv[v]]


In [58]:
LT_i = {i:nodos[i]["LT"] for i in nodos.keys()} #lower bounds ventana de tiempo por nodo
RT_i = {i:nodos[i]["RT"] for i in nodos.keys()} #upper bounds ventana de tiempo por nodo


In [59]:
T_ijv={} # tiempo de entre i y j (tiempo de viaje + tiempo de operación en i)
for i in nodos.keys():
    T_ijv[i] = {} 
    for j in nodos.keys():
        T_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                T_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                T_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Tiempo_Viaje_hrs']
                if 0<i<=n_cargos and (i,j) in Av[v]: 
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Origen_hrs']
                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    T_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Tiempo_Destino_hrs']
                
                # se les asigna tiempo de operación igual a 0 a los (i,j) que no pertenecen a Av
                # los (i,j) que no pertenecen a Av aún tienen el tiempo entre el puerto de i y j

In [60]:
C_ijv={} # costo de entre i y j (costo de viaje + costo de operación en i)
for i in nodos.keys():
    C_ijv[i] = {} 
    for j in nodos.keys():
        C_ijv[i][j] = {} 
        for v in V:
            cargo_i= nodos[i]["ID_Cargo"]
            cargo_j = nodos[j]["ID_Cargo"]
            if i>n_cargos*2+len(V) or j>n_cargos*2+len(V):
                C_ijv[i][j][v] = 0
            else: 
                puerto_origen = nodos[i]["ID_Puerto"]
                puerto_destino = nodos[j]["ID_Puerto"]
                C_ijv[i][j][v] = costos_transporte_dict[v,puerto_origen,puerto_destino]['Costo_Viaje_libras']
                if i==11 and j==2 and i==1:
                    print(C_ijv[i][j][v])
                if 0<i<=n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Origen_hrs']

                if n_cargos*2>=i>n_cargos and (i,j) in Av[v]:
                    C_ijv[i][j][v] += costos_tiempo_dict[v,cargo_i]['Costos_Destino_hrs']

In [61]:
Qj={i:nodos[i]["Tamano"] for i in nodos.keys()}

Cs = {cargos_sheet[i]['ID_Cargo']:cargos_sheet[i]['Costo_SPOT'] for i in cargos_sheet.keys()}

In [62]:
nodos_index = [i for i in nodos.keys()]

In [63]:
def obj_rule(model):
    return sum(C_ijv[i][j][v]*model.x[i,j,v]  for v in V for (i,j) in Av[v])+sum(Cs[i]*model.y[i] for i in nodos_carga)

def const_1(model, i):
    #return sum(model.x[i,j,v]  for v in V for j in Nv[v]) + model.y[i] == 1
    return sum(model.x[i,j,v]  for v in V for j in Nv[v] if i in Nv[v]) + model.y[i] == 1

def const_2(model, v):
    #return sum(model.x[o[v],j,v] for j in Nv[v]) == 1
    return sum(model.x[o[v],j,v] for j in Nv[v] if o[v]!=j) == 1

def const_3(model, i, v):
    if i in Nv[v]:
        if i!=o[v] and i!=d[v]:
            #return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[j,i,v] for j in Nv[v] if j!=i)==0
            return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[j,i,v] for j in Nv[v])==0
    return Constraint.Skip

def const_4(model, v):
    #return sum(model.x[j,d[v],v] for j in Nv[v]) == 1
    return sum(model.x[j,d[v],v] for j in Nv[v] if d[v]!=j) == 1

def const_5(model, i, j, v):
    if j in NP_v[v]:
        if (i,j) in Av[v]:
            return model.l[i,v]+Qj[j]-model.l[j,v]<=Kv[v]*(1-model.x[i,j,v])
    return Constraint.Skip

def const_6(model, i, j, v):
    if j in NP_v[v]:
        if (i,n_cargos+j) in Av[v]:
            return model.l[i,v]-Qj[j]-model.l[n_cargos+j,v]<=Kv[v]*(1-model.x[i,n_cargos+j,v])
    return Constraint.Skip

def const_7(model, i, v):
    if i in NP_v[v]:
        return (0, model.l[i,v], Kv[v])
    return Constraint.Skip

def const_8(model, i, j, v):
    if (i,j) in Av[v]:
        return model.t[i,v]+T_ijv[i][j][v]-model.t[j,v]<=(RT_i[i]+T_ijv[i][j][v])*(1-model.x[i,j,v])
    return Constraint.Skip

def const_9(model, i, v):
    if i in NP_v[v]:
        #return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[n_cargos+i,j,v] for j in Nv[v] if j!=i+n_cargos)==0
        return sum(model.x[i,j,v] for j in Nv[v])-sum(model.x[n_cargos+i,j,v] for j in Nv[v])==0
    return Constraint.Skip

def const_10(model, i, v):
    if i in NP_v[v]:
        return model.t[i,v]+T_ijv[i][n_cargos+i][v]-model.t[n_cargos+i,v]<=0
    return Constraint.Skip
    
def const_11(model, i, v):
    if i in Nv[v]:
        return (LT_i[i], model.t[i,v], RT_i[i])
    return Constraint.Skip

In [64]:
def definir_modelo():
    #crear modelo
    model_1 = ConcreteModel()
    #definir variables 
    model_1.x = Var(nodos_index, nodos_index, V, within=Binary)
    model_1.y = Var(nodos_carga, within=Binary)
    model_1.t = Var(nodos_index,V)
    model_1.l = Var(nodos_index,V, within=NonNegativeReals)
    #definir VO
    model_1.obj = Objective(rule=obj_rule)
    #definir restricciones
    model_1.const_1 = Constraint(nodos_carga, rule=const_1)
    model_1.const_2 = Constraint(V, rule=const_2)
    model_1.const_3 = Constraint(nodos_index, V, rule=const_3)
    model_1.const_4 = Constraint(V, rule=const_4)
    model_1.const_5 = Constraint(nodos_index, nodos_index, V, rule=const_5)
    model_1.const_6 = Constraint(nodos_index, nodos_index, V, rule=const_6)
    model_1.const_7 = Constraint(nodos_index, V, rule=const_7)
    model_1.const_8 = Constraint(nodos_index, nodos_index, V, rule=const_8)
    model_1.const_9 = Constraint(nodos_index, V, rule=const_9)
    model_1.const_10 = Constraint(nodos_index, V, rule=const_10)
    model_1.const_11 = Constraint(nodos_index, V, rule=const_11)
    return model_1

In [65]:
def random_removal(q, modelo):
    print("entré a la función")
    nuevo_modelo = definir_modelo() #se define un nuevo modelo
    cargos_list = random.sample([i for i in range(1, n_cargos+1)],q) #lista de cargos totales
    for i in nodos_index:
        for j in nodos_index:
            for v in V:
                if modelo.x[i,j,v]() == 1 and not(i in cargos_list) and not(j in cargos_list) and (i<=n_cargos*2 and j<=n_cargos*2):
                    print(f"seteo una x({i},{j},{v})",modelo.x[i,j,v]())
                    #los barcos que llevaban cargos siguen llevando,
                    # a menos que el cargo que llevaban fuera uno de los elegidos
                    nuevo_modelo.x[i,j,v].setlb(val = 1)
                    nuevo_modelo.x[i,j,v].setub(val = 1)
    for i in range(1, n_cargos+1):
        if modelo.y[i]() == 1 and not(i in cargos_list):
            print("seteo una y")
            #los spot que llevaban cargos siguien llevando,
            #a menos que el cargo que llevaban fuera uno de los elegidos
            nuevo_modelo.y[i].setlb(val = 1)
            nuevo_modelo.y[i].setub(val = 1)
    return nuevo_modelo

In [66]:
def creation(modelo, solver):
    solver.solve(modelo)
    return modelo
    

In [69]:
import random
import time
count=0
modelo = definir_modelo()
q = 3
print("hola")
for i in range(n_cargos):
    modelo.y[i+1].setlb(val = 1)
    modelo.y[i+1].setub(val = 1)
solver = SolverFactory('gurobi')
solver.solve(modelo)
print(modelo.x[1,1,1]()==1)
print("solver")
start_time = time.time()
while(count<2):
    modelo = random_removal(q,modelo)
    modelo = creation(modelo,solver)
    print(modelo.obj())
    print("--- %s seconds ---" % (time.time() - start_time))
    count += 1

hola
False
solver
entré a la función
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
46550630.0
--- 225.663733959198 seconds ---
entré a la función
seteo una x(45.0,105.0,7) 1.0
seteo una x(58.0,118.0,6) 1.0
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y
seteo una y


In [72]:
for i in modelo.x:
    if modelo.x[i].value == 1:
        print(i)

(11.0, 71.0, 9)
(71.0, 144, 9)
(24.0, 84.0, 10)
(84.0, 145, 10)
(43.0, 103.0, 8)
(103.0, 143, 8)
(45.0, 105.0, 7)
(105.0, 142, 7)
(58.0, 118.0, 6)
(118.0, 141, 6)
(121, 136, 1)
(122, 137, 2)
(123, 138, 3)
(124, 139, 4)
(125, 140, 5)
(126, 58.0, 6)
(127, 45.0, 7)
(128, 43.0, 8)
(129, 11.0, 9)
(130, 24.0, 10)
(131, 146, 11)
(132, 147, 12)
(133, 148, 13)
(134, 149, 14)
(135, 150, 15)
